In [39]:
#choose model 
import glob
import pandas as pd
datadir = '/home/dmoi/datasets/PPI_antileakage/'
pairs = glob.glob(f'{datadir}*.txt')
dfs = { p.split('/')[-1].split('.txt')[0]: pd.read_csv(p, sep = ' ', header=None, names=['protA', 'protB']) for p in pairs if 'filtered' not in p }
for k in dfs.keys():
	print(k, dfs[k].shape)
#compile non-redundant set of proteins
proteins = set()
for k in dfs.keys():
	proteins = proteins.union( set(dfs[k]['protA'].tolist()) ).union( set(dfs[k]['protB'].tolist()) )
print(f'Number of unique proteins: {len(proteins)}')


Intra2_neg_rr (26024, 2)
Intra1_neg_rr (81596, 2)
Intra2_pos_rr (26024, 2)
Intra0_pos_rr (29630, 2)
Intra0_neg_rr (29630, 2)
Intra1_pos_rr (81596, 2)
Number of unique proteins: 11019


In [40]:

#create a struct folder
folder = '/home/dmoi/datasets/PPI_antileakage/structures/'
import os
if not os.path.exists(folder):
	os.mkdir(folder)

#download structures
from foldtree2.src import AFDB_tools
import tqdm
for struct in tqdm.tqdm(proteins):
	AFDB_tools.grab_struct( struct, folder )

#encode structures

#transformer to weighted set

#weighted jacccard pairise



  2%|▊                                     | 230/11019 [00:00<00:10, 996.24it/s]

error downloading structure for Q8N2C7 HTTP Error 404: Not Found
https://alphafold.ebi.ac.uk/files/AF-Q8N2C7-F1-model_v4.pdb
error downloading structure for Q03001 HTTP Error 404: Not Found
https://alphafold.ebi.ac.uk/files/AF-Q03001-F1-model_v4.pdb
error downloading structure for O95714 HTTP Error 404: Not Found
https://alphafold.ebi.ac.uk/files/AF-O95714-F1-model_v4.pdb
error downloading structure for O75962 HTTP Error 404: Not Found
https://alphafold.ebi.ac.uk/files/AF-O75962-F1-model_v4.pdb


  5%|█▉                                   | 583/11019 [00:00<00:05, 1851.73it/s]

error downloading structure for P22105 HTTP Error 404: Not Found
https://alphafold.ebi.ac.uk/files/AF-P22105-F1-model_v4.pdb
error downloading structure for Q8WXI7 HTTP Error 404: Not Found
https://alphafold.ebi.ac.uk/files/AF-Q8WXI7-F1-model_v4.pdb
error downloading structure for Q8NCM8 HTTP Error 404: Not Found
https://alphafold.ebi.ac.uk/files/AF-Q8NCM8-F1-model_v4.pdb
error downloading structure for Q8WZ42 HTTP Error 404: Not Found
https://alphafold.ebi.ac.uk/files/AF-Q8WZ42-F1-model_v4.pdb


  7%|██▌                                  | 766/11019 [00:00<00:08, 1226.14it/s]

...

error downloading structure for Q92813 HTTP Error 404: Not Found
https://alphafold.ebi.ac.uk/files/AF-Q92813-F1-model_v4.pdb
error downloading structure for Q9UKN1 HTTP Error 404: Not Found
https://alphafold.ebi.ac.uk/files/AF-Q9UKN1-F1-model_v4.pdb
error downloading structure for O60494 HTTP Error 404: Not Found
https://alphafold.ebi.ac.uk/files/AF-O60494-F1-model_v4.pdb
error downloading structure for Q8NFC6 HTTP Error 404: Not Found
https://alphafold.ebi.ac.uk/files/AF-Q8NFC6-F1-model_v4.pdb


 98%|██████████████████████████████████▍| 10828/11019 [00:10<00:00, 1146.97it/s]

error downloading structure for P49792 HTTP Error 404: Not Found
https://alphafold.ebi.ac.uk/files/AF-P49792-F1-model_v4.pdb
error downloading structure for Q9HC84 HTTP Error 404: Not Found
https://alphafold.ebi.ac.uk/files/AF-Q9HC84-F1-model_v4.pdb
error downloading structure for Q86UP3 HTTP Error 404: Not Found
https://alphafold.ebi.ac.uk/files/AF-Q86UP3-F1-model_v4.pdb
error downloading structure for Q15911 HTTP Error 404: Not Found
https://alphafold.ebi.ac.uk/files/AF-Q15911-F1-model_v4.pdb


100%|███████████████████████████████████| 11019/11019 [00:10<00:00, 1017.16it/s]

error downloading structure for Q70CQ2 HTTP Error 404: Not Found
https://alphafold.ebi.ac.uk/files/AF-Q70CQ2-F1-model_v4.pdb


In [41]:
#check which structures were downloaded
print('Checking downloaded structures...')
#clear filtered csv files
filtered = glob.glob(f'{datadir}*_filtered.txt')
for f in filtered:
	os.remove(f)
#recompile non-redundant set of proteins
print( f'Number of unique proteins to check: {len(proteins)}' )
downloaded = glob.glob(f'{folder}*.pdb')
downloaded = [ d.split('/')[-1].split('.pdb')[0] for d in downloaded ]
print(f'Number of downloaded structures: {len(downloaded)}')
#intersect with proteins
proteins = proteins.intersection( set(downloaded) )
print(f'Number of structures with PDB files: {len(proteins)}')
#filter dfs
for k in dfs.keys():
	print( f'Filtering {k} with {len(proteins)} proteins...' )
	print( k, dfs[k].shape )
	dfs[k] = dfs[k][ dfs[k]['protA'].isin(proteins) & dfs[k]['protB'].isin(proteins) ]
	print(k, dfs[k].shape)
	#add label column
	if 'neg' in k:
		dfs[k]['label'] = 0
	else:
		dfs[k]['label'] = 1
	#add dataset label
	dfs[k]['dataset'] = k.split('_')[0]
	#save filtered dfs
	dfs[k].to_csv(f'{datadir}{k}_filtered.txt', sep=' ', header=False, index=False)

#concatenate all filtered dfs
all_pairs = pd.concat( [ dfs[k] for k in dfs.keys() ] )
all_pairs = all_pairs.drop_duplicates().reset_index(drop=True)


Checking downloaded structures...
Number of unique proteins to check: 11019
Number of downloaded structures: 10851
Number of structures with PDB files: 10851
Filtering Intra2_neg_rr with 10851 proteins...
Intra2_neg_rr (26024, 2)
Intra2_neg_rr (25564, 2)
Filtering Intra1_neg_rr with 10851 proteins...
Intra1_neg_rr (81596, 2)
Intra1_neg_rr (77714, 2)
Filtering Intra2_pos_rr with 10851 proteins...
Intra2_pos_rr (26024, 2)
Intra2_pos_rr (25566, 2)
Filtering Intra0_pos_rr with 10851 proteins...
Intra0_pos_rr (29630, 2)
Intra0_pos_rr (29017, 2)
Filtering Intra0_neg_rr with 10851 proteins...
Intra0_neg_rr (29630, 2)
Intra0_neg_rr (28995, 2)
Filtering Intra1_pos_rr with 10851 proteins...
Intra1_pos_rr (81596, 2)
Intra1_pos_rr (77719, 2)


In [52]:
print( all_pairs['label'].value_counts() )
print( all_pairs['dataset'].value_counts() )

valdf = all_pairs[all_pairs['dataset'] == 'Intra0']
traindf = all_pairs[all_pairs['dataset'] == 'Intra1']
testdf = all_pairs[all_pairs['dataset'] == 'Intra2']
print( valdf['label'].value_counts() )
print( traindf['label'].value_counts() )
print( testdf['label'].value_counts() )

label
1    132302
0    132273
Name: count, dtype: int64
dataset
Intra1    155433
Intra0     58012
Intra2     51130
Name: count, dtype: int64
label
1    29017
0    28995
Name: count, dtype: int64
label
1    77719
0    77714
Name: count, dtype: int64
label
1    25566
0    25564
Name: count, dtype: int64


In [53]:
cd /home/dmoi/projects/foldtree2/foldtree2

/home/dmoi/projects/foldtree2/foldtree2


In [54]:
from torch_geometric.data import DataLoader
from foldtree2.src.pdbgraph import PDB2PyG
converter = PDB2PyG( aapropcsv='./config/aaindex1.csv')
downloaded = glob.glob(f'{folder}*.pdb')
#converter.store_pyg(downloaded, filename='structs_pairwise.h5', foldxdir = None, verbose = False)

In [71]:
from foldtree2.src.pdbgraph import StructureDataset
import torch

# Create StructureDataset from the stored PyG file
struct_dat = StructureDataset('structs_pairwise.h5')

# Create a custom loader that yields pairs of encoded structures
class PairwiseStructureLoader:
	def __init__(self, struct_dataset, pairs_df):
		self.struct_dataset = struct_dataset
		self.pairs_df = pairs_df
		# Create mapping from protein ID to dataset index
		self.prot_to_idx = {prot_id: idx for idx, prot_id in enumerate(struct_dataset.structlist)}
	
	def __getitem__(self, idx):
		row = self.pairs_df.iloc[idx]
		prot_a, prot_b = row['protA'], row['protB']
		if prot_a in self.prot_to_idx and prot_b in self.prot_to_idx:
			struct_a = self.struct_dataset[self.prot_to_idx[prot_a]]
			struct_b = self.struct_dataset[self.prot_to_idx[prot_b]]
			y = torch.tensor( row['label'] ) if 'label' in row else None
			return { 'struct_a': struct_a, 'struct_b': struct_b, 'label': y }
		else:
			raise IndexError("Protein ID not found in structure dataset.")

	def __iter__(self):
		for _, row in self.pairs_df.iterrows():
			prot_a, prot_b = row['protA'], row['protB']
			if prot_a in self.prot_to_idx and prot_b in self.prot_to_idx:
				struct_a = self.struct_dataset[self.prot_to_idx[prot_a]]
				struct_b = self.struct_dataset[self.prot_to_idx[prot_b]]
				y = torch.tensor( row['label'] ) if 'label' in row else None
				yield { 'struct_a': struct_a, 'struct_b': struct_b, 'label': y }

	def __len__(self):
		return len(self.pairs_df)


In [72]:

# Create loaders for each dataset split
pairwise_loaders = {}
for split_name, split_dfs in [('train', traindf), ('valid', valdf), ('test', testdf)]:
	pairwise_loaders[split_name] = {}
	pairwise_loaders[split_name]['dataset'] = PairwiseStructureLoader(struct_dat, split_dfs)
	print("Pairwise structure loaders created for:")
	print(f"  {split_name}: {len(split_dfs)} pairs")
	#create dataloaders
	pairwise_loaders[split_name]['dataloader'] = DataLoader(pairwise_loaders[split_name]['dataset'], batch_size=32, shuffle=(split_name=='train'), num_workers=4)
	print(f"  Dataloader created with {len(pairwise_loaders[split_name]['dataloader'])} batches")
	print( pairwise_loaders[split_name] )
	
	


Pairwise structure loaders created for:
  train: 155433 pairs
  Dataloader created with 4858 batches
{'dataset': <__main__.PairwiseStructureLoader object at 0x7f58de56c920>, 'dataloader': <torch_geometric.deprecation.DataLoader object at 0x7f59d0100560>}
Pairwise structure loaders created for:
  valid: 58012 pairs
  Dataloader created with 1813 batches
{'dataset': <__main__.PairwiseStructureLoader object at 0x7f59d0101a60>, 'dataloader': <torch_geometric.deprecation.DataLoader object at 0x7f59d0101250>}
Pairwise structure loaders created for:
  test: 51130 pairs
  Dataloader created with 1598 batches
{'dataset': <__main__.PairwiseStructureLoader object at 0x7f59d0100920>, 'dataloader': <torch_geometric.deprecation.DataLoader object at 0x7f59d0102ae0>}


In [73]:
#print a sample from the pairwise loader
for sample in pairwise_loaders['train']['dataloader']:
	print(sample)
	break

{'struct_a': HeteroDataBatch(
  identifier=[32],
  AA={
    x=[19799, 20],
    batch=[19799],
    ptr=[33],
  },
  R_true={
    x=[19799, 3, 3],
    batch=[19799],
    ptr=[33],
  },
  bondangles={
    x=[19799, 3],
    batch=[19799],
    ptr=[33],
  },
  coords={
    x=[19799, 3],
    batch=[19799],
    ptr=[33],
  },
  fourier1di={
    x=[19799, 80],
    batch=[19799],
    ptr=[33],
  },
  fourier1dr={
    x=[19799, 80],
    batch=[19799],
    ptr=[33],
  },
  fourier2di={
    x=[32, 1300],
    batch=[32],
    ptr=[33],
  },
  fourier2dr={
    x=[32, 1300],
    batch=[32],
    ptr=[33],
  },
  godnode={
    x=[32, 5],
    batch=[32],
    ptr=[33],
  },
  godnode4decoder={
    x=[32, 5],
    batch=[32],
    ptr=[33],
  },
  plddt={
    x=[19799, 1],
    batch=[19799],
    ptr=[33],
  },
  positions={
    x=[19799, 256],
    batch=[19799],
    ptr=[33],
  },
  res={
    x=[19799, 857],
    batch=[19799],
    ptr=[33],
  },
  t_true={
    x=[19799, 3],
    batch=[19799],
    ptr=[33],
 

In [ ]:
#load encoder model
encoder_modelpath = '/home/dmoi/projects/foldtree2/models/encoder_epoch_90.pt'
import torch
encoder_model = torch.load(encoder_modelpath)
modelname = encoder_modelpath.split('/')[-1].split('.pt')[0]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
encoder_model = encoder_model.to(device)
encoder_model.eval()
model_embeddings = encoder_model.vector_quantizer.embeddings.clone().detach().cpu()

In [ ]:
#use the encoder to transform the structures to ords 


z1, _  = encoder_model.forward( sample['struct_a'].to(device) )
z2, _  = encoder_model.forward( sample['struct_b'].to(device) )
#discretize
z1_q , _ = encoder_model.vector_quantizer.discretize_z( z1 )
z2_q , _ = encoder_model.vector_quantizer.discretize_z( z2 )
#embeddings 
z1_e = encoder_model.vector_quantizer.embeddings[z1_q]
z2_e = encoder_model.vector_quantizer.embeddings[z2_q]



In [ ]:
#create the pairwise transformer model
pairwise_model = pairwise.PairwiseTransformerModel(
    input_dim=model_embeddings.shape[1],
    num_heads=8,
    num_layers=6,
    ff_dim=2048,
    dropout=0.1,
    encoder=encoder_model
)
#initialize the model embeddings with the encoder embeddings 
pairwise_model.embeddings = model_embeddings
pairwise_model = pairwise_model.to(device)
pairwise_model.train()

In [ ]:
#forward pass
figurestack = []
reload = False
import glob
if reload == True:
	pairwise_models = glob.glob('models/pairwise_epoch_*.pth')
	if len(pairwise_models) > 0:
		latest_model = max(pairwise_models, key=os.path.getctime)
		print(f"Loading model from {latest_model}")
		pairwise_model = torch.load(latest_model)
def pairloss(preds, labels):
	loss_fn = torch.nn.BCEWithLogitsLoss()
	return loss_fn(preds, labels.float())
	


	#save model checkpoint
	if epoch % 20 == 0:
		torch.save(encoder, f'models/encoder_epoch_{epoch}.pt')
		torch.save(decoder, f'models/decoder_epoch_{epoch}.pt')





In [ ]:

#extract the encoder embedding dictionary
encoder_codebook = encoder.vector_quantizer.embeddings.detach().clone().cpu()
#show the embedding dictionary
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
plt.imshow(encoder_codebook, aspect='auto', cmap='viridis')
plt.colorbar()
plt.title('Encoder Embedding Dictionary')
plt.xlabel('Embedding Dimension')
plt.ylabel('Code Index')
plt.show()

In [ ]:
def load_encoded_fasta(filename, alphabet=None, replace=None):
	seqstr = ''
	seqdict = {}

	with open(filename, 'r') as f:

		#read all chars of file into a string
		for i,line in enumerate(tqdm.tqdm(f)):

			if line[0] == '>' and i > 0:
				seqdict[ID] = seqstr[:-1]
				ID = line[1:].strip()
				seqstr = ''
			elif line[0] == '>' and i == 0:
				ID = line[1:].strip()
				seqstr = ''
			embeddingselse:
				seqstr += line
		if '' in seqdict:
			del seqdict['']
		
	encoded_df = pd.DataFrame( seqdict.items() , columns=['protid', 'seq'] )
	encoded_df['seqlen'] = encoded_df.seq.map( lambda x: len(x) )
	#change index to protid
	encoded_df.index = encoded_df.protid
	encoded_df = encoded_df.drop( 'protid', axis=1 )
	encoded_df['ord'] = encoded_df.seq.map( lambda x: [ ord(c) for c in x] )
	#hex starts at 1
	encoded_df['hex2'] = encoded_df.ord.map( lambda x: [ hex(c) for c in x] )
	return encoded_df

	

In [ ]:
encoded_fasta = modelname+'_aln_encoded.fasta' 
encoded_df = load_encoded_fasta(encoded_fasta, alphabet=encoder.alphabet, replace=False)
print(encoded_df.head())
encoded_df['ord'] = encoded_df.seq.map( lambda x: [ ord(c) for c in x] )
# get the set of ords
ordset = set()
for o in encoded_df.ord:
	ordset = ordset.union( set(o) )
print(f'Number of unique ords: {len(ordset)}')
#map ords to indices
ord2idx = { o:i for i,o in enumerate(sorted(list(ordset))) }
print(ord2idx)
#map indices to ords
idx2ord = { i:o for o,i in ord2idx.items() }
print(idx2ord)
encoded_df['idx'] = encoded_df.ord.map( lambda x: [ ord2idx[c] for c in x] )
print(encoded_df.head())
#use the indices to train the pairwise model


In [ ]:
#encode structures
traindfs = { k: dfs[k] for k in dfs.keys() if 'Intra0' in k }
validfs = { k: dfs[k] for k in dfs.keys() if 'Intra1' in k }
testdfs = { k: dfs[k] for k in dfs.keys() if 'Intra2' in k }

In [ ]:

for data in pairwise_loaders['train']['PPI']:
	prot1, prot2 = data['struct_a'], data['struct_b']
	y = data['label']
	prot1 = prot1.to(device)
	prot2 = prot2.to(device)
	y = y.to(device) if y is not None else None
	#forward pass through encoder
	z, vqloss = encoder_model( prot1 , None )
	zdiscrete = encoder_model.vector_quantizer.discretize_z(z)
	data['ords'] = zdiscrete.detach().to(device)

In [ ]:
# add the encoded sequences to the dataframes
for k in traindfs.keys():
	traindfs[k] = traindfs[k].merge( encoded_df, left_on='protA', right_index=True, how='inner' )
	traindfs[k] = traindfs[k].merge( encoded_df, left_on='protB', right_index=True, how='inner', suffixes=('_A', '_B') )
	
	print(k, traindfs[k].shape)
	

In [ ]:
#train the model for pairwise

from foldtree2.src import pairwise





In [ ]:
#pr and roc

In [ ]:
#compare to hashed jaccard 

#pr and roc